In [33]:
## 1. Import Thư Viện Cần Thiết
import numpy as np
import pandas as pd
import pickle
import os


In [34]:
## 2. Tải Mô Hình và Dữ Liệu
try:
    # Tải model.pkl
    with open('model.pkl', 'rb') as f:
        model_components = pickle.load(f)
    print("Đã tải file model.pkl thành công.")
    
    # Gán các thành phần
    cosine_sim = model_components['cosine_sim_matrix']
    indices = model_components['song_indices']
    df_model = model_components['df_model']
except FileNotFoundError:
    print("Lỗi: Không tìm thấy file model.pkl. Vui lòng chạy notebook 3 trước.")
    exit()
except Exception as e:
    print(f"Lỗi khi tải hoặc giải nén model.pkl: {e}")
    exit()

Đã tải file model.pkl thành công.


In [36]:
## 3. Định Nghĩa Hàm Gợi Ý (get_recommendations)

def get_recommendations(title, cosine_sim_matrix, df_data, indices_series, top_n=10):
    """
    Hàm này lấy tiêu đề bài hát và trả về top N bài hát tương tự nhất.
    """
    
    # 1. Kiểm tra tiêu đề có trong dữ liệu không
    if title not in indices_series:
        print(f"Lỗi: Bài hát '{title}' không có trong cơ sở dữ liệu mô hình.")
        return pd.DataFrame() 

    # 2. Lấy chỉ mục (index) của bài hát đầu vào
    idx = indices_series[title]
    
    # 3. Lấy tất cả điểm tương đồng
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    
    # 4. Sắp xếp các bài hát dựa trên điểm tương đồng (giảm dần)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 5. Lấy điểm của TOP N+1 bài hát (Bỏ qua bài hát đầu tiên - chính nó)
    sim_scores = sim_scores[1:top_n+1]
    
    # 6. Lấy chỉ mục của các bài hát được gợi ý
    song_indices = [i[0] for i in sim_scores]
    
    # 7. Xây dựng DataFrame kết quả
    recommendations = df_data.iloc[song_indices].copy()
    
    # Thêm điểm tương đồng vào kết quả
    recommendations['Similarity_Score'] = [i[1] for i in sim_scores]
    
    return recommendations[['Title', 'Artist', 'Genre', 'Similarity_Score']]


## 4. Đánh Giá Thực Tế (Chọn một bài hát mẫu)

# --- GIẢI PHÁP FAILSAFE: CHỌN BÀI HÁT ĐẦU TIÊN ---
# Lấy tiêu đề bài hát đầu tiên trong DataFrame. Việc này đảm bảo sample_title là một STRING đơn lẻ.
sample_title = df_model['Title'].iloc[0]

print(f"\n--- ĐÁNH GIÁ MÔ HÌNH: Gợi ý cho Bài hát Đầu vào ---")
print(f"Bài hát đầu vào: **{sample_title}**")

# Gọi hàm gợi ý
recommended_songs = get_recommendations(
    title=sample_title, 
    cosine_sim_matrix=cosine_sim, 
    df_data=df_model, 
    indices_series=indices, 
    top_n=10
)

# In kết quả
if not recommended_songs.empty:
    print("\nTop 10 Bài hát được gợi ý:")
    print(recommended_songs)
else:
    print("\nKhông thể thực hiện gợi ý.")


## 5. Kết Luận
print("\n--- KẾT LUẬN ---")
print("Quá trình đánh giá mô hình gợi ý đã hoàn tất.")


--- ĐÁNH GIÁ MÔ HÌNH: Gợi ý cho Bài hát Đầu vào ---
Bài hát đầu vào: **Xa Em Buồn - Yvonne**

Top 10 Bài hát được gợi ý:
                           Title     Artist             Genre  \
84    Xa Em Lãng Mạn - Christina  Trúc Nhân            Ballad   
128             Ngày Ấy Lãng Mạn  Trúc Nhân            Ballad   
977                      Đêm Vui  Trúc Nhân            Ballad   
1674                Mưa Tàn Phai  Trúc Nhân            Ballad   
720          Lặng Thinh Lãng Mạn  Trúc Nhân            Ballad   
402             Sài Gòn Tàn Phai  Trúc Nhân            Ballad   
1176                 Ngày Ấy Đẹp  Trúc Nhân            Ballad   
911                  Sài Gòn Vui  Trúc Nhân            Ballad   
1799   Cuộc Sống Ngọt Ngào - Roy  Trúc Nhân            Ballad   
572                     Mưa Buồn  Trúc Nhân  R&B/Hip-Hop Việt   

      Similarity_Score  
84            0.884018  
128           0.882849  
977           0.871094  
1674          0.840747  
720           0.821651  
402         